# TNT Cates (Living Polymers): Creep Response

## Objectives
- Fit TNTCates model to creep compliance data
- Analyze UCM-like creep response: J(t) = 1/G_0 + t/(G_0*tau_d)
- Extract effective relaxation time tau_d from linear creep
- Show compliance governed by tau_d
- Perform Bayesian inference with NUTS

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from scipy.integrate import cumulative_trapezoid

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_creep,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

## Theory: Cates Model for Creep

The Cates model predicts creep compliance following step stress:

**Creep compliance (UCM-like):**
In the fast-breaking limit:
$$J(t) = \frac{1}{G_0} + \frac{t}{G_0 \tau_d} + \frac{t}{\eta_s}$$

where:
- $1/G_0$: Elastic compliance (instantaneous)
- $t/(G_0 \tau_d)$: Viscoelastic creep (linear in time)
- $t/\eta_s$: Viscous flow (steady state)

**Effective relaxation time:**
$$\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$$

**Physical interpretation:**
- Faster breaking → shorter $\tau_d$ → faster creep
- Slope of $J(t)$ at intermediate times gives $1/(G_0 \tau_d)$

## Load Creep Data and Convert to Strain

In [ ]:
time_data, shear_rate, stress_applied = load_ml_ikh_creep(stress_pair_index=0)

strain = cumulative_trapezoid(shear_rate, time_data, initial=0)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Applied stress: {stress_applied:.2f} Pa")
print(f"Strain range: {strain.min():.2e} to {strain.max():.2e}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, strain, 'o', label='Strain data', markersize=4)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain γ', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Creep Strain', fontsize=14)

compliance = strain / stress_applied
ax2.loglog(time_data, compliance, 'o', label='J(t)', markersize=4)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_title('Creep Compliance', fontsize=14)

plt.tight_layout()
display(fig)
plt.close(fig)

## NLSQ Fitting

In [ ]:
model = TNTCates()

start_time = time.time()
model.fit(time_data, strain, test_mode='creep', stress=stress_applied, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
strain_pred_fit = model.predict(time_data, test_mode='creep', stress=stress_applied)
quality = compute_fit_quality(strain, strain_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
time_pred = jnp.linspace(time_data.min(), time_data.max(), 300)
strain_pred = model.predict(time_pred, test_mode='creep', stress=stress_applied)

# Compute residuals
residuals = strain - model.predict(time_data, test_mode='creep', stress=stress_applied)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(time_data, strain, 'o', label='Data', markersize=4)
ax1.plot(time_pred, strain_pred, '-', linewidth=2, label='TNTCates fit')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain γ', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title('Creep Strain Fit', fontsize=14)

ax2.plot(time_data, residuals, 'o', markersize=4)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Residuals', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_title('Residuals', fontsize=14)

plt.tight_layout()
display(fig)
plt.close(fig)

## Physical Analysis: Extract tau_d from Compliance

In [ ]:
tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

J_elastic = 1.0 / nlsq_params['G_0']
J_slope = 1.0 / (nlsq_params['G_0'] * tau_d)

compliance_pred = strain_pred / stress_applied

mask = (time_pred > tau_d/10) & (time_pred < tau_d*10)
if jnp.sum(mask) > 10:
    log_t = jnp.log(time_pred[mask])
    log_J = jnp.log(compliance_pred[mask])
    slope_fit = jnp.polyfit(log_t, log_J, 1)[0]
else:
    slope_fit = 0.0

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")
print(f"\nCompliance Components:")
print(f"  Elastic compliance (1/G_0): {J_elastic:.4e} Pa^-1")
print(f"  Viscoelastic slope (1/(G_0*tau_d)): {J_slope:.4e} Pa^-1 s^-1")
print(f"  Log-log slope near tau_d: {slope_fit:.2f} (expected ~1 for linear creep)")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: UCM-like creep behavior")
else:
    print(f"\n  → Full Cates dynamics: May deviate from simple UCM")

## Visualize Compliance Components

In [ ]:
J_elastic_line = J_elastic * np.ones_like(time_pred)
J_viscous_line = J_elastic + time_pred * J_slope

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, compliance, 'o', label='Data', markersize=4, zorder=3)
ax.loglog(time_pred, compliance_pred, '-', linewidth=2, label='TNTCates fit', zorder=2)
ax.loglog(time_pred, J_elastic_line, '--', linewidth=1.5, label='Elastic (1/G₀)', zorder=1)
ax.loglog(time_pred, J_viscous_line, ':', linewidth=1.5, label='Elastic + Viscous', zorder=1)
ax.axvline(tau_d, color='gray', linestyle='-.', alpha=0.7, label=f'τ_d = {tau_d:.2e}s')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa$^{-1}$)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Compliance Components', fontsize=14)
display(fig)
plt.close(fig)

## Bayesian Inference with NUTS

In [ ]:
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Starting Bayesian inference with NUTS...")
print(f"Warmup: {NUM_WARMUP}, Samples: {NUM_SAMPLES}, Chains: {NUM_CHAINS}")

start_time = time.time()
bayesian_result = model.fit_bayesian(
    time_data, strain,
    test_mode='creep',
    stress=stress_applied,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    seed=42
)
bayes_time = time.time() - start_time

print(f"\nBayesian inference completed in {bayes_time:.2f} seconds")
print(f"Time per sample: {bayes_time/(NUM_WARMUP + NUM_SAMPLES):.3f} seconds")

## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Trace Plot

In [ ]:
idata = az.from_dict(posterior={k: v.reshape(NUM_CHAINS, NUM_SAMPLES) for k, v in posterior.items()})

fig = az.plot_trace(idata, var_names=param_names, compact=False, figsize=(12, 8))
plt.suptitle('MCMC Trace Plots', fontsize=14, y=1.0)
plt.tight_layout()
display(fig)
plt.close(fig)

## ArviZ Pair Plot

In [ ]:
fig = az.plot_pair(
    idata,
    var_names=param_names,
    kind='kde',
    marginals=True,
    figsize=(10, 10)
)
plt.suptitle('Posterior Correlations', fontsize=14, y=0.995)
display(fig)
plt.close(fig)

## ArviZ Forest Plot

In [ ]:
fig = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(8, 4)
)
plt.suptitle('95% Credible Intervals', fontsize=14)
display(fig)
plt.close(fig)

## Posterior Predictive Distribution

In [ ]:
n_posterior_samples = 200
sample_indices = np.random.choice(NUM_SAMPLES * NUM_CHAINS, n_posterior_samples, replace=False)

predictions = []
for idx in sample_indices:
    # Set model parameters from posterior sample
    for j, name in enumerate(param_names):
        model.parameters.set_value(name, float(posterior[name].flatten()[idx]))
    # Predict with updated parameters
    pred_i = model.predict(time_pred, test_mode='creep', stress=stress_applied)
    predictions.append(np.array(pred_i))

predictions = np.array(predictions)
pred_mean = np.mean(predictions, axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, strain, 'o', label='Data', markersize=4, zorder=3)
ax.plot(time_pred, pred_mean, '-', linewidth=2, label='Posterior mean', zorder=2)
ax.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% CI', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain γ', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Posterior Predictive Distribution', fontsize=14)
display(fig)
plt.close(fig)

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']
J_slope_posterior = 1.0 / (posterior['G_0'] * tau_d_posterior)

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")
print(f"  Viscoelastic slope: {np.mean(J_slope_posterior):.4e} ± {np.std(J_slope_posterior):.4e} Pa^-1 s^-1")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(J_slope_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(J_slope_posterior), color='r', linestyle='--', label='Mean')
ax2.set_xlabel(r'$1/(G_0 \tau_d)$ (Pa$^{-1}$ s$^{-1}$)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Viscoelastic Slope Distribution', fontsize=12)

plt.tight_layout()
display(fig)
plt.close(fig)

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "creep", param_names)

## Key Takeaways

1. **UCM-like creep** in Cates model: $J(t) = 1/G_0 + t/(G_0 \tau_d)$
2. **Effective relaxation time** $\tau_d$ governs viscoelastic compliance slope
3. **Linear creep region** (around $\tau_d$) provides direct measurement of $\tau_d$
4. **Compliance decomposition** separates elastic, viscoelastic, and viscous contributions
5. **Bayesian inference** quantifies uncertainty in compliance parameters from creep data

**Next steps:** Compare $\tau_d$ from creep with values from other protocols (relaxation, SAOS) to verify consistency.